## Tectonics and Landscape Evolution Project



In [12]:
from landlab import RasterModelGrid 

import pandas as pd
import numpy as np
from FieldWork_class import FieldWork

#Import you field work data
data = pd.read_csv('/Users/poliveira/Projects_PhD/tect_land_project/field_work_tatun_park.csv')
data

#Calculate parameters for your model
fw = FieldWork(data)
result = fw.calculate_dip_strike()

#Set the size of your area
w = 1000
l  = 1000
cell_resolution = 30

rows,cols = fw.calculate_grid_size(w,l,cell_resolution) 

#create grid
my_grid = RasterModelGrid((rows,cols),10)


#####run to steady state- fake model 

k_sp = 5E-5

uplift = 5e-5



The average strike is 10.0
The average dip is 58.904761904761905
Your area have 1000x1000, the grid has 33.333333333333336 rows and 33.333333333333336 columns
